[LearnOpenGL-PBR](https://learnopengl.com/PBR/Lighting)

# Theory
**PBR**(**Physically Based Rendering**)是一系列基于物理的渲染技术.但**PBR**只是基于物理对现实光照的近似.一个**PBR**光照模型需要满足以下三个条件才能称作基于物理:
- 基于微表面理论
- 能量守恒
- 使用基于物理的**BRDF**

以下章节将专注于由**Disney**首次提出并由**Epic Games**为实时渲染所改进的**PBR**方法, 并实现以下的视觉效果.![](./Images/ibl_specular_result_textured.png)

### The microfacet model
所有的**PBR**技术都基于微表面模型:任何表面都可以描述为无数微小镜面,基于表面粗糙度不同,这些微平面的对齐方式存在很大差异.![](./Images/microfacets.png)

表面越粗糙, 微平面排列越混乱,入射光线更容易沿不同的方向散射, 从而导致更广泛的镜面反射.相反的,在光滑表面上光线更倾向于向相似的方向反射, 从而产生更小更锐利的反射效果: ![](./Images/microfacets_light_rays.png)

从微观程度上来讲,没有任何表面是完全光滑的,在给定的粗糙度参数下可以从统计学角度近似计算微表面粗糙度:即可以近似计算与向量$h$对齐的微平面的比率.向量$h$是光线方向$l$和视线方向$v$的半角向量: $$h=\frac{l+v}{\|l+v\|}$$

微平面与半角向量的对齐程度越高,高光反射越锐利和强烈.在$0-1$的粗糙度参数范围内,可以估算出微平面的对齐比率: ![](./Images/ndf.png)
可以看出,更高的粗糙度表现出了更大的高光反射范围,相反的,光滑表面的高光反射范围更小更锐利.

### Energy conservation
微平面理论引入了一种能量守恒:出射光能量不得超过入射光能量(不包括自发光表面).从上图来看,随着粗糙度增加,高光反射区域加大,但亮度随之减小了. 如果假设每个像素点上的高光反射强度都相同(不管高光发射的区域有多大),更粗糙的表面会反射更多的能量,这是违反能量守恒的.这也是为什么在光滑表面高光反射更强而在粗糙表面更暗.光能量分为漫反射和高光反射.当一束光线击中一个表面,将会被分解成折射(**refraction**)部分和反射部分.反射部分不会被表面吸收的,即高光反射.余下的折射部分会进入表面并被吸收,也就是漫反射.被折射的光不会直接被表面吸收:从物理上讲,光线可以模拟成一束持续向前传递直到完全损耗的能量.而光束损失能量的方式就是碰撞.每种材质都由可以与光线碰撞的细小粒子组成(如下图所示),这些粒子在每次与光线碰撞时吸收部分或全部光能并转化为热能. ![](./Images/surface_reaction.png)

- 通常光能不会被完全吸收,并且光线会从它与其他粒子碰撞的点开始持续向任意方向散射直到能量耗尽或离开表面,从表面反射出的光线展现了表面的颜色.但在基于物理的渲染中,会假设所有的折射光线都在一个很小的碰撞区域内被吸收和散射,忽略了散射光线会在一定距离离开表面带来的影响.
- 被称作次表面散射的着色技术会将这些计算在内以极大的提升材质视觉质量,但伴随而来的自然是以牺牲性能为代价.
- 对金属表面来说,折射和散射有一个额外的细微差别:金属表面与非金属(**绝缘体**)表面对光的反应不同.金属表面虽然遵从同样的反射和折射准则,但所有的折射光会被直接吸收.也就是说金属表面只有高光反射,不会表现出漫反射颜色.因此,金属与非金属表面在**PBR**管线中会被区别对待.

由此可以得出:反射与折射是互斥的,任何反射的光能都不会再被材质所吸收,因此,光的总能量就等于反射光的能量与进入表面的折射光的能量之和.首先计算出高光反射部分在入射光能量中所占的百分比,按照能量守恒,就可以直接计算出折射光部分所占的比率:
```cpp
float kS = calculateSpecularComponent(...); // reflection/specular fraction
float kD = 1.0 - kS;                 // refraction/diffuse  fraction
```

### The reflectance equation(反射率方程)
基于物理的渲染严格遵循一个特定的称为反射率方程的渲染方程. 
$$L_{o}(p, \omega_{o})=\int\limits_{\Omega} f_{r}(p, \omega_{i}, \omega_{o})L_{i}(p, \omega_{i})n \cdot \omega_{i}d\omega_{i}$$ 
- 辐射率(**radiance**)$L$:用来量化单一方向光的强度.
- 辐射通量(**Radiant flux**)$\phi$:用瓦特来度量的光源传输的能量.光是不同波长能量的集合,每种波长与一种特定的颜色相关.因此光传输的能量可看做与它各种波长相关的函数.通常波长位于390纳米至700纳米之间的属于可见光谱范围,即可被人眼所观察到的波长范围.下图展示了日光中不同波长的光所表示的能量. ![](./Images/daylight_spectral_distribution.png) 辐射通量即为不同波长的函数所围成的面积.在计算中通常采用**RGB**三原色(**light color**)而不是变化的波长函数来简化表示辐射通量.这种编码方式虽然会损失很多信息,但对于视觉效果来说是微不足道的.
- 立体角(**Solid angle**)$\omega$:表示投影到单位球体上的截面面积.![](./Images/solid_angle.png) 想象站在单位球球心看向这个截面所在的方向, 截面投影的轮廓大小就是立体角.
- 辐射强度(**Radiant intensity**): 每单位上的辐射通量,或者说光源穿过单位球体上的投影区域时传输的能量.例如说, 存在一个全向光源向所有方向均匀辐射能量,辐射强度就表示光在单位立体角上的能量. ![](./Images/radiant_intensity.png) 辐射强度的公式表示为: $$I=\frac{d\phi}{d\omega}$$ 即 $I$ 是立体角$\omega$上的辐射通量$\phi$.

综上,辐射率方程即辐射强度为$\phi$的光源在单位面积$A$,单位立体角$\omega$上辐射的能量: $$L=\frac{d^2\phi}{dAd\omega cos\theta}$$ ![](./Images/radiance.png) 

辐射率是一定区域内光量的辐射量单位,它受到入射光与表面法线夹角的影响:光线辐射到表面的程度越小,光线越弱,而当光线垂直于表面的时候最强.这与漫反射光照相似,$\cos\theta$等于光线方向与表面法线的点乘: 
```cpp
float cosTheta = dot(lightDir, N);
```

如果将立体角$\omega$和单位面积$A$看做无限小,那么就可以用辐射率来度量空间中一束光线穿过某一个点时的辐射通量,从而计算出单束光线对单个点(像素)的辐射率.实际计算中,将立体角$\omega$转换为方向向量$\omega$,面积$A$转换为点$p$,我们就可以在shader中用辐射率公式直接计算出单束光线对于每个像素的强度.

实际上,当讨论辐射率的时候,通常需要考虑点$p$上所有的入射光,即辐射率的总和,称为辐照度(**irradiance**).再来看反射率方程: 
$$L_{o}(p, \omega_{o})=\int\limits_{\Omega} f_{r}(p, \omega_{i}, \omega_{o})L_{i}(p, \omega_{i})n \cdot \omega_{i}d\omega_{i}$$

其中的$L$表示某个点$p$的辐射率,并且某个无限小的入射立体角$\omega_{i}$可以当做一个入射向量.光线相对表面入射角度的余弦$\cos\theta$是光能量的缩放因子,在反射率方程中表示为$n\cdot \omega_{i}$.反射率方程计算了点$p$在方向$\omega_{o}$(相对于观察者来说的出射方向)上反射出来的辐射量总和$L_{o}(p, \omega)$. 换句话说:$L_{o}$表示了从$\omega_{o}$观察点$p$时,光线反射的辐照度之和.

由于反射率方程是基于辐照度的,即光的所有入射辐射率总和,所以这不仅仅是针对单个的入射方向,而是要考量以点$p$为中心的半球$\Omega$内所有方向的入射光.这个半球可以表示为以表面法线为中心轴线的半个球体. ![](./Images/hemisphere.png)

在反射率方程中即以$\int$表示的半球区域$\Omega$内来自所有方向$d\omega_{i}$的能量总和.积分表示某个函数曲线所表示的面积,其计算结果是解析解或者数值解.然而渲染方程和反射率方程都没有解析解,用数值法求解这个积分的过程可表示为将反射率方程在半球域$\Omega$内分解为一定步长的多个离散解然后按照步长求平均值.这一方法称为黎曼和(**Riemann sum**),可用伪码表示如下:
```cpp
int steps = 100;
float sum = 0.0f;
vec3 P    = ...;
vec3 Wo   = ...;
vec3 N    = ...;
float dW  = 1.0f / steps;
for(int i = 0; i < steps; ++i) 
{
    vec3 Wi = getNextIncomingLightDir(i);
    sum += Fr(P, Wi, Wo) * L(P, Wi) * dot(N, Wi) * dW;
}
```
通过缩放$dW$,所求得的总和将等于被积分方程的面积或体积.$dW$等同于反射率方程中的$d\omega_{i}$.在数学上,$d\omega_{i}$是用来计算积分的一个连续的符号,与代码中的$dW$并没有直接的联系(因为它是黎曼和中的一个离散步长).采用离散的步长能够求得被积分函数总面积的一个近似值,可以通过增加求解步数来提高黎曼和的精度.

反射率方程表示了光线$L_{o}$从观察者方向上击中点$p$的所有半球$\Omega$区域内所有入射方向$\omega_{i}$被$f_{r}$所约束的辐射率的和.入射光辐射率可以来自光源或者一张表示所有入射光辐射率的环境贴图(**IBL**).现在,唯一的未知量就是符号$f_{r}$,即**BRDF**或称作双向反射分布函数,它是基于表面材质属性的入射光辐射率权重.

### BRDF
**BRDF**以入射光方向$\omega_{i}$,出射光方向(观察方向)$\omega_{o}$,表面法线$n$以及微表面粗糙度参数$\alpha$为输入参数,近似的求出每束光线$\omega_{i}$在给定的材质属性下对最终的反射光线的贡献程度.例如,如果表面是完全光滑的(比如镜子),对于所有的入射光线$\omega_{i}$,除了和反射光线$\omega_{o}$同向的光线**BRDF**将返回$1.0$外,其余的**BRDF**将都将返回$0.0$.

**BRDF**基于微表面理论近似的表示材质的反射和折射属性.一个物理上可信的**BRDF**必须满足能量守恒定律,即反射光能量的总和不能超过入射光能量.严格上来说,同样采用$\omega_{i}$和$\omega_{o}$作为输入的**Blinn-Phong**也可看做是**BRDF**,但它没有考虑能量守恒准则,因此不能认为是符合物理的.目前已经有几种基于物理的**BRDF**可以模拟表面对于光的反应,但几乎所有的实时**PBR**渲染管线都使用**Cook-Torrance BRDF**.
**Cook-Torrance BRDF**包含漫反射和镜面反射两个部分:$$f_{r}=k_{d}f_{lambert}+k_{s}f_{cook-torrance}$$
这里$k_{d}$是入射光能量中被折射的部分所占的比率,$k_{s}$是被反射部分的比率.**BRDF**的左侧是以$f_{lambert}$表示的漫反射部分,也称为**Lambertian diffuse**, 与漫反射着色中使用的常量因子相似:$$f_{lambert}=\frac{c}{\pi}$$
$c$表示反射率或表面颜色.除以$\pi$是为了对漫反射光进行标准化,$\pi$是之前含有**BRDF**的积分方程的缩放因子.

目前存在多种不同的看似真实的模型可表示**BRDF**的漫反射部分,但相应的运算开销也较大.但依照**Epic**给出的结论,**Lambertian diffuse**已经足以满足多数实时渲染的需求了.
**Cook-Torrance BRDF**模型的镜面反射部分要更复杂些: $$f_{CookTorrance}=\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)}$$

**Cook-Torrance BRDF**镜面反射由三个函数和一个标准化因子组成.字母**D**,**F**,**G**分别表示近似计算表面反射属性特定部分的函数:
- 法线分布函数**D**: 估算表面上与半角向量方向一致的微表面的数量,它受表面粗糙度影响,是模拟微表面的主要函数.
- 几何函数**G**: 描述微表面的自阴影属性.当表面相对粗糙的时候,表面上的微表面会相互遮挡从而减少表面反射.
- 菲涅尔方程**F**: 描述表面在不同角度下的反射比率.

以上每种函数都是用来估算其所对应的物理量的,并且都有多个不同的版本,有些较为真实,有些效率更高.**Epic Games**的**Brian Karis**对于多种不同的估算方法做了大量的[研究](http://graphicrants.blogspot.nl/2013/08/specular-brdf-reference.html).我们在此采用与**UE4**相同的函数,即
- **D: Trowbridge-Reitz GGX**, 
- **F: Fresnel-Schlick approximation**
- **G: Smith's Schlick-GGX**

#### Normal distribution function
法线分布函数**D**从统计学上估算了表面上与半角向量$h$方向一致的微表面所占的比率.通过给定的一些粗糙度参数可以得到很多统计学上估算微表面取向度的**NDF**, 在此使用其中一种称为**Trowbridge-Reitz GGX**的函数: $$NDF_{GGXTR}(n,h,\alpha)=\frac{\alpha^2}{\pi((n\cdot h)^2(\alpha^2-1)+1)^2}$$, 其中:
    - $h$是半角向量
    - $\alpha$是表面粗糙度参数
在不同的粗糙度系数下可以得到如下的视觉效果:![](./Images/ndf.png)
可以看出,当表面粗糙度较低时(即表面比较光滑),与半角向量方向一致的微表面会集中在一个很小的半径内,由此**NDF**会产生出一个高亮点.在一个粗糙的表面上,微表面的取向更加随机,因此与半角向量方向一致的微表面分布在一个大得多的半径范围内,从而产生一个更加灰暗的显示效果.

**Trowbridge-Reitz GGX**转化为**GLSL**如下:
```cpp
float DistributionGGX(vec3 N, vec3 H, float a)
{
    float a2     = a*a;
    float NdotH  = max(dot(N, H), 0.0);
    float NdotH2 = NdotH*NdotH;
	
    float nom    = a2;
    float denom  = (NdotH2 * (a2 - 1.0) + 1.0);
    denom        = PI * denom * denom;
	
    return nom / denom;
}
```

#### Geometry function
几何函数从统计学上估算了表面上的微表面相互遮挡的比例.![](./Images/geometry_shadowing.png)
与**NDF**相似,几何函数采用了材质的粗糙度参数作为输入,表面越粗糙微表面间相互遮蔽的概率越大.在此使用称为**Schlick-GGX**的函数,它是**GGX**和**Schlick-Beckmann approximation**的结合: $$G_{SchlickGGX}(n,v,k)=\frac{n\cdot v}{(n\cdot v)(1-k)+k}$$
这里的$k$是粗糙度$\alpha$的重映射,并针对直接光和**IBL**有不同的表示:
$$k_{direct}=\frac{(\alpha + 1)^2}{8}$$ 
$$k_{IBL}=\frac{\alpha ^ 2}{2}$$
注意:根据引擎对粗糙度表示方法的差异$\alpha$值可能会有所不同.
为了有效地估算几何部分,需要考虑视线方向(几何遮蔽**geometry obstruction**)和光线方向(几何阴影**geometry shadowing**),而**Smith**方法就满足这一需求:$$G(n,v,l,k)=G_{sub}(n,v,k)G_{sub}(n,l,k)$$
使用**Smith**方法和**Schlick-GGX**作为$G_{sub}$,在不同的粗糙度$R$下可以呈现以下的视觉效果:![](./Images/geometry.png).
几何函数是一个值域为$[0.0, 1.0]$的乘数,其中1.0(或白色)表示没有微平面阴影,0.0(或黑色)表示微平面完全被遮蔽.
几何函数写成**GLSL**如下:
```cpp
float GeometrySchlickGGX(float NdotV, float k)
{
    float nom  = NdotV;
    float denom = NdotV * (1.0 - k) + k;
	
    return nom / denom;
}
  
float GeometrySmith(vec3 N, vec3 V, vec3 L, float k)
{
    float NdotV  = max(dot(N, V), 0.0);
    float NdotL  = max(dot(N, L), 0.0);
    float ggx1   = GeometrySchlickGGX(NdotV, k);
    float ggx2   = GeometrySchlickGGX(NdotL, k);
	
    return ggx1 * ggx2;
}
```

#### Fresnel equation
菲涅尔方程描述了光线被反射部分和被折射部分的比率,这个比率随观察表面的视角变化而变化:当光线击中表面的时候,菲涅尔方程根据表面与视线的夹角来表示光线被反射的百分比.基于反射率和能量守恒,可以直接获得光线被折射部分的能量.
当在垂直视角上观察时,每一个表面或者材质都有一个基础反射率,但当从某个角度看向平面时,所有的反射都会变得更加明显(相对于表面的基础反射率来说).当从一个完全垂直的角度(**90度**)观察时,所有的表面理论上可以完全反射光线,这一现象因菲涅尔而闻名并由菲涅尔方程所描述.
菲涅尔方程是一个相当复杂的函数,可采用**Fresnel-Schlick**来近似:$$F_{Schlick}(h,v,F_{0})=F_{0}+(1-F_{0})(1-(h\cdot v))^5$$
其中$F_{0}$表示表面的基础反射率,它是由一种称为折射指数(**Indices of Refraction**或**IOR**)计算出来的.正如从一个球形表面可以看到的那样,观察方向与表面掠射角越接近(即半角向量与表面夹角接近**90度**),菲涅尔现象越明显,反射也越强:![](./Images/fresnel.png).

**Fresnel-Schlick**近似仅仅对电介质或非金属表面有定义.对于导体或金属表面,使用**IOR**计算基础反射率并不能得到正确的结果,所以需要对导体使用不同的菲涅尔方程,但为了方便,可以预先计算出表面在法向入射角(**normal incidence**)(即从**0度角**方向看向表面)的反应$F_{0}$以用于之后的近似运算,之后基于视角方向的**Fresnel-Schlick**近似的对这个值进行插值,这样就可以对金属和非金属统一用一种公式了.

表面对法向入射的反应或者说基础反射率可以在一些大型[数据库](http://refractiveindex.info)中找到,从**Naty Hoffman**的讲义中找到一些常用的值列表如下:

Material|$F_{0}$(Linear)|$F_{0}(sRGB)$|Color|
:--:|:--:|:--:|:--:|
Water|(0.02, 0.02, 0.02)|(0.15, 0.15, 0.15)|<td style="background-color: #737373"></td>
Plastic/Glass (Low)|(0.03, 0.03, 0.03)|(0.21, 0.21, 0.21)|<td style="background-color: #363636"></td>
Plastic High|(0.05, 0.05, 0.05)|(0.24, 0.24, 0.24)|<td style="background-color: #3D3D3D"></td>
Glass (high)/Ruby|(0.08, 0.08, 0.08)|(0.31, 0.31, 0.31)|<td style="background-color: #4F4F4F"></td>
Diamond|(0.17, 0.17, 0.17)|(0.45, 0.45, 0.45)|<td style="background-color: #737373"></td>
Iron|(0.56, 0.57, 0.58)|(0.77, 0.78, 0.78)|<td style="background-color: #C5C8C8"></td>
Copper|(0.95, 0.64, 0.54)|(0.98, 0.82, 0.76)|<td style="background-color: #FBD2C3"></td>
Gold|(1.00, 0.71, 0.29)|(1.00, 0.86, 0.57)|<td style="background-color: #FFDC92"></td>
Aluminium|(0.91, 0.92, 0.92)|(0.96, 0.96, 0.97)|<td style="background-color: #F6F6F8"></td>
Silver|(0.95, 0.93, 0.88)|(0.98, 0.97, 0.95)|<td style="background-color: #FBF8F3"></td>

可以观察到所有的绝缘体其基础反射率都不会高于$0.17$,这其实是例外而不是一般规则,导体的基础反射率起始值更高并大多在0.5至1.0之间变化.此外,导体或金属表面的基础反射率都是彩色的.这也是为什么$F_{0}$用**RGB**三原色来表示(法向上的反射率会随波长变化),这种现象只能在金属表面观察到.

金属表面现对于电介质(绝缘体)表面的这种特殊属性引入了所谓的金属工作流.在这种金属工作流中我们需要一个称为金属度(**metalness**)的参数来描述一个表面金属还是非金属.

- 理论上,材质的金属度应该是非此即彼的:即要么是金属要么不是,不应该有中间状态.但大部分渲染管线都允许为表面设置一个介于0.0到1.0之间的线性值.这主要是因为材质贴图不够精确,例如,拥有细小沙粒,灰尘的金属表面很难通过一个二元的金属度值来渲染.

用这一预先为电介质和导体计算的$F_{0}$可以为这两种类型的表面用同一种**Fresnel-Schlick**近似,但对于金属表面还需要为基础反射率着色,通常表示如下:
```cpp
vec3 F0 = vec3(0.04);
F0    = mix(F0, surfaceColor.rgb, metalness); ///lerp
```

对电介质表面,$F_{0}$取大部分常见电介质的平均值.$0.04$的基础反射率已经足够覆盖大部分电介质并在不借助其他表面参数的情况下产生物理可信的效果.然后,基于金属的表面属性,要么采用电介质的基础反射率要么采用$F_{0}$作为表面颜色.由于金属表面吸收了全部的折射光,它们没有漫反射颜色,所以直接采用表面贴图颜色来作为它们的基础反射率.

**Fresnel Schlick**近似在代码中表示如下:
```cpp
vec3 fresnelSchlick(float cosTheta, vec3 F0)
{
    return F0 + (1.0 - F0) * pow(1.0 - cosTheta, 5.0);
}
```
其中**cosTheta**是表面法线$n$和半角向量$h$的点积.

### Cook-Torrance reflectance equation
基于以上所有的**Cook-Torrance BRDF**分量, 可以得到以下最终的反射率方程来表示基于物理的**BRDF**:
$$L_{o}(p, \omega_{o})=\int\limits_{\Omega} (k_{d}\frac{c}{\pi}+k_{s}\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)})L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$

这个方程不完全是数学上正确的.由于**BRDF**的高光部分已经隐式的包含了比率$k_{s}$, 故可以得到最终的反射率方程:
$$L_{o}(p, \omega_{o})=\int\limits_{\Omega} (k_{d}\frac{c}{\pi}+\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)})L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$

这个方程完整的描述了基于物理的渲染模型也就是一般意义上的基于物理的渲染(**PBR**).

### Authoring PBR materials
**PBR**管线所需的每一个表面参数都可以用贴图来定义.利用贴图可以逐片元的控制特定的表面对于光线的反应:不管采样点是金属的,粗糙的或者光滑的,也不论表面如何响应不同波长的光.
下面是一列经常在**PBR**管线中出现的贴图及它们所表现出的视觉效果:![](./Images/textures.png)

   - Albedo: 反射率贴图指定表面任一图素的颜色或基础反射率.这与之前使用的漫反射贴图非常类似,不同的是所有的光照信息都是从贴图上提取出来的.漫反射贴图中经常包含微小的阴影或深色的裂纹,而在反射率贴图中不需要这些,反射率贴图应该只包含表面颜色或者说折射吸收系数(**refracted absorption coefficients**)
   - Normal: 法线贴图允许逐像素的指定一个唯一的法线,以此可以为表面产生高低起伏的视觉效果.
   - Metallic: 金属度贴图逐像素的指定一个图素是否是金属.根据**PBR**引擎的设置,**artists**可以将金属度表示为灰度值或非黑即白的二元值.
   
   - Roughness: 粗糙度贴图在逐像素维度指定表面的粗糙程度.采样出的粗糙度值表示统计学上微表面的取向.更粗糙的表面会产生范围更广,更模糊的反射,而光滑表面会产生更集中,更清晰的反射.一些**PBR**引擎采用光滑度贴图而不是粗糙度贴图,这对于一些**artists**来说更直观,这些值在采样时被转化为$(1.0-smoothness)$.
   
   - AO: 环境光遮蔽或AO贴图为表面及其周围的几何图形指定了一个额外的阴影因子.例如一个砖块表面,反射率贴图在砖块的缝隙部分不应该有任何阴影信息,而AO贴图则应该指定这些光线难以溢出的暗边.在光照计算的最后附加上环境光遮蔽能大福提升场景视觉效果.AO贴图是手动制作或在模型软件中预结算出来的.
   
### Further reading
- [Background](https://blog.selfshadow.com/publications/s2013-shading-course/hoffman/s2013_pbs_physics_math_notes.pdf): Physics and Math of Shading by Naty Hoffmann: there is too much theory to fully discuss in a single article so the theory here barely scratches the surface; if you want to know more about the physics of light and how it relates to the theory of PBR this is the resource you want to read.
- [Real shading in Unreal Engine 4](https://blog.selfshadow.com/publications/s2013-shading-course/karis/s2013_pbs_epic_notes_v2.pdf): discusses the PBR model adopted by Epic Games in their 4th Unreal Engine installment. The PBR system we'll focus on in these chapters is based on this model of PBR.
- [[SH17C] Physically Based Shading, by knarkowicz](https://www.shadertoy.com/view/4sSfzK): great showcase of all individual PBR elements in an interactive ShaderToy demo.
- [Marmoset: PBR Theory](https://www.marmoset.co/toolbag/learn/pbr-theory): an introduction to PBR mostly meant for artists, but nevertheless a good read.
- [Coding Labs: Physically based rendering](http://www.codinglabs.net/article_physically_based_rendering.aspx): an introduction to the render equation and how it relates to PBR.
- [Coding Labs: Physically Based Rendering - Cook–Torrance](http://www.codinglabs.net/article_physically_based_rendering_cook_torrance.aspx): an introduction to the Cook-Torrance BRDF.
- [Wolfire Games - Physically based rendering](http://blog.wolfire.com/2015/10/Physically-based-rendering): an introduction to PBR by Lukas Orsvärn.
- [[SH17C] Physically Based Shading](https://www.shadertoy.com/view/4sSfzK): a great interactive shadertoy example (warning: may take a while to load) by Krzysztof Narkowi showcasing light-material interaction in a PBR fashion.


# Lighting
反射率$L$表示光源在给定的立体角$\omega$下的辐射通量$\phi$.假设立体角$\omega$无限小,这样辐射率就表示光源在单一光线或方向上的辐射通量.

假设存在一个点光源(向所有方向发出均匀亮度的光源),它的辐射通量用**RGB**表示是$(23.47,21.31,20.79)$,则它在所有方向上的辐射强度都等于它的辐射通量.当对表面上指定的一个点$p$着色时,它的半球领域$\Omega$内所有可能的入射光中只有一个入射方向直接来自于点光源,由于场景中只有一个光源,则对于表面上的点$p$,所有其他可能的入射光方向上的辐射度为0:![](./Images/lighting_radiance_direct.png)

首先,假设点光源没有光线衰减,即无论把光源放到哪一个位置,入射光的辐射率都相同(除去作为辐射率缩放因子的入射角$cos\theta$).由于不管从哪个角度看,点光源的辐射强度都相同,因此可以用一个常量向量$(23.47,21.31,20.79)$表示它的辐射通量.

更一般的:用辐射率方程$L$度量直接点光源光照颜色,并根据光源到点$p$的距离衰减,$n\cdot \omega_{i}$为缩放因子,转换为代码如下:
```cpp
vec3  lightColor  = vec3(23.47, 21.31, 20.79);
vec3  wi          = normalize(lightPos - fragPos);
float cosTheta    = max(dot(N, Wi), 0.0);
float attenuation = calculateAttenuation(fragPos, lightPos);
vec3  radiance    = lightColor * attenuation * cosTheta;
```
#### Linear and HDR rendering
目前为止的计算都假定是在线性空间中的, 所以必须在最后进行[**gamma correct**](https://learnopengl.com/Advanced-Lighting/Gamma-Correction).**PBR**要求所有的输入都必须是线性的,否则将得到不正确的结果.为了尽可能的让光照的输入值接近它们的真实物理属性,其辐射率或颜色值可能会在较大的光谱值范围内波动,$L_{o}$的值会很快超出**LDR**的表示范围,在**gamma correction**前必须通过**tone/exposure map**将**HDR**映射到**LDR**.
```cpp
/// Reinhard operator
color = color / (color + vec3(1.0));
color = pow(color, vec3(1.0/2.2)); 
```
![](./Images/lighting_linear_vs_non_linear_and_hdr.png)

#### Full direct lighting PBR shader
```cpp
#version 330 core
out vec4 FragColor;
in vec2 TexCoords;
in vec3 WorldPos;
in vec3 Normal;

// material parameters
uniform vec3  albedo;
uniform float metallic;
uniform float roughness;
uniform float ao;

// lights
uniform vec3 lightPositions[4];
uniform vec3 lightColors[4];

uniform vec3 camPos;

const float PI = 3.14159265359;
  
float DistributionGGX(vec3 N, vec3 H, float roughness);
float GeometrySchlickGGX(float NdotV, float roughness);
float GeometrySmith(vec3 N, vec3 V, vec3 L, float roughness);
vec3 fresnelSchlick(float cosTheta, vec3 F0);

void main()
{		
    vec3 N = normalize(Normal);
    vec3 V = normalize(camPos - WorldPos);

    vec3 F0 = vec3(0.04); 
    F0 = mix(F0, albedo, metallic);
	           
    // reflectance equation
    vec3 Lo = vec3(0.0);
    for(int i = 0; i < 4; ++i) 
    {
        // calculate per-light radiance
        vec3 L = normalize(lightPositions[i] - WorldPos);
        vec3 H = normalize(V + L);
        float distance    = length(lightPositions[i] - WorldPos);
        float attenuation = 1.0 / (distance * distance);
        vec3 radiance     = lightColors[i] * attenuation;        
        
        // cook-torrance brdf
        float NDF = DistributionGGX(N, H, roughness);        
        float G   = GeometrySmith(N, V, L, roughness);      
        vec3 F    = fresnelSchlick(max(dot(H, V), 0.0), F0);       
        
        vec3 kS = F;
        vec3 kD = vec3(1.0) - kS;
        kD *= 1.0 - metallic;	  
        
        vec3 numerator    = NDF * G * F;
        float denominator = 4.0 * max(dot(N, V), 0.0) * max(dot(N, L), 0.0);
        vec3 specular     = numerator / max(denominator, 0.001);  
            
        // add to outgoing radiance Lo
        float NdotL = max(dot(N, L), 0.0);                
        Lo += (kD * albedo / PI + specular) * radiance * NdotL; 
    }   
  
    vec3 ambient = vec3(0.03) * albedo * ao;
    vec3 color = ambient + Lo;
	
    color = color / (color + vec3(1.0));
    color = pow(color, vec3(1.0/2.2));  
   
    FragColor = vec4(color, 1.0);
} 
```
采用这个**fragment shader**,在四个点光源作用下,分别调整不同球体的金属度和粗糙度值,可以得到如下的效果:![](./Images/lighting_result.png)
从下到上金属度的值变化范围为$0.0-1.0$,粗糙度值则从左到右从$0.0$增长至$1.0$.([完整示例](https://learnopengl.com/code_viewer_gh.php?code=src/6.pbr/1.1.lighting/lighting.cpp))

<iframe src="https://oneshader.net/embed/6b8a7c6363" style="width:80%; height:440px; border:0;margin-left:10.0%; margin-right:12.5%;" frameborder="0" allowfullscreen></iframe>

#### Textured PBR
- **albedo**贴图通常是在**sRGB**空间制作
- **ambient occlusion**贴图通常也需要从**sRGB**空间转换到线性空间
- **metallic**和**roughness**贴图通常是在线性空间中制作的
- 金属表面在直接光环境下通常看起来较暗,因为它们没有漫反射.
![](./Images/lighting_textured.png)

# IBL

### Diffuse irradiance
**IBL**是将周围环境当做一个大光源的一系列光照技术的集成.通常实现为对**cubemap**进行采样,即将**cubemap**中的每个像素点当做一个光源.

由于**IBL**会捕捉部分或全部环境光,故通常认为它是一种更精确的环境光照输入,甚至是全局光照的粗略近似.

来自周围环境每一个入射方向$\omega_{i}$的入射光都有可能有辐射度贡献,这为求解辐射度积分提出两点要求:
- 需要从任意给定的方向$\omega_{i}$检索场景辐射度的方法
- 必须能够实时计算积分

第一个要求相对简单:给定一个**cubemap**,我们可以将**cubemap**上的每一个图素当做一个光源,在$\omega_{i}$方向上采样该**cubemap**即是场景在这个方向上的辐射度.
```cpp
vec3 radiance = texture(_cubemapEnvironment, w_i).rgb;
```
但是,求解积分需要从所有可能的$\omega_{i}$方向上在半球区域$\Omega$内对环境贴图进行采样,这对**fragment shader**调用来说代价昂贵.为了更高效的求解积分必须进行预计算.辐射率方程中可以看出,**BRDF**的漫反射项$k_{d}$和高光项$k_{s}$是相互独立的,因此可以将反射率方程分为两部分:
$$L_{o}(p, \omega_{o})=\int\limits_{\Omega} (k_{d}\frac{c}{\pi})L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}+\int\limits_{\Omega} (k_{s}\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)})L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$

漫反射积分中,**diffuse lambert**项是常量(即反射率$c$,折射比例$k_{d}$和$\pi$都是常量),因此可以移到积分外部:$$L_{o}(p, \omega_{o})=k_{d}\frac{c}{\pi}\int\limits_{\Omega} L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$

此积分仅依赖于$\omega_{i}$(假设$p$是环境贴图的中心),因此可以利用卷积(**convolution**)预计算出一张新的存储各个采样方向$\omega_{o}$上漫反射积分项的**cubemap**.

为了计算环境贴图卷积,需要在半球区域$\omega$内对入射方向$\omega{i}$大量离散采样,然后计算它们的平均辐射度,以此来求解每个采样方向$\omega_{o}$上的积分.![](./Images/ibl_hemisphere_sample.png)
这一预计算的**cubemap(irradiance map)**可以看做表面在$\omega_{o}$方向上所有的间接漫反射光辐射度之和.
- 辐射率方程也依赖位置$p$,即之前假定的**irradiance map**中心,这就是说所有的间接漫反射光都必须来自同一张环境贴图,这可能会影响光照效果的真实感(特别是在室内场景).渲染引擎通常通过在场景中放置**reflection probe**来解决这一问题:每个**reflection probe**只计算它周围的环境光,位置$p$的漫反射辐射度即为离它最近的**reflection probes**之间的插值.
下图是环境贴图以及它的**irradiance map**的示例:![](./Images/ibl_irradiance.png)
**irradiance map**看起来像是环境贴图的平均颜色,从任一方向采样这张图将得到该方向上场景的辐照度.

### PBR and HDR
灯泡和太阳光的差异是显著的,因此只有在**HDR**渲染环境中才可能区分每种光照的强度.目前为止使用的**cubemap**都属于**LDR**,如果将它们当做物理输入参数是不对的.

### The radiance HDR file format
**HDR**格式允许存储超过$1.0$的颜色,它不是每个通道存储$32$位的数据,而是每个通道存储$8$位的数据,然后用**alpha**通道存储指数(虽然有一定的精度损失).[sIBL archive](http://www.hdrlabs.com/sibl/archive.html)上有一些免费**HDR**环境图.
![](./Images/ibl_hdr_radiance.png)
上面的这张环境贴图是从一个球体投影到平面上的,以此可以将环境信息存储到一张称为**equirectangular map**的图中,由于大部分视觉信息都是存储在水平方向,只有少量是在底部和顶部方向,因此这种方式对于大部分渲染器来说是一个不错的折中方案.

### From Equirectangular to Cubemap
直接从**equirectangular map**获取环境信息相对于直接采样**cubemap**来说依然是代价昂贵的,可以将**equirectangular map**转为**cubemap**:渲染一个立方体,并将**equirectangular map**投影到立方体的六个面.在**vertex shader**中仅把顶点传递到**fragment shader**并当做**3D**采样向量.
```cpp
#version 330 core
layout (location = 0) in vec3 aPos;

out vec3 localPos;

uniform mat4 projection;
uniform mat4 view;

void main()
{
    localPos = aPos;  
    gl_Position =  projection * view * vec4(localPos, 1.0);
}
```
在**fragment shader**中采样**equirectangular map**并输出:
```cpp
#version 330 core
out vec4 FragColor;
in vec3 localPos;

uniform sampler2D equirectangularMap;

const vec2 invAtan = vec2(0.1591, 0.3183);
vec2 SampleSphericalMap(vec3 v)
{
    vec2 uv = vec2(atan(v.z, v.x), asin(v.y));
    uv *= invAtan;
    uv += 0.5;
    return uv;
}

void main()
{		
    vec2 uv = SampleSphericalMap(normalize(localPos)); // make sure to normalize localPos
    vec3 color = texture(equirectangularMap, uv).rgb;
    
    FragColor = vec4(color, 1.0);
}
```
如果将这一结果渲染到场景中心将得到如下的结果:![](./Images/ibl_equirectangular_projection.png)
将最终所得的**cubemap**渲染到之前的球体场景中将得到如下的结果(直接将**HDR**图渲染到**LDR**):![](./Images/ibl_hdr_environment_mapped.png)

### Cubemap convolution
在方向$\omega_{i}$上采样**HDR**环境贴图可以得到场景在该方向上的辐射度$L(p,\omega_{i})$,为了求解积分,必须对半球区域$\Omega$内所有可能方向进行采样,但可能的方向理论上是无限的,因此只能有限的进行等间隔或随机采样来获取辐照度的一个相对准确的近似值,从而离散的求解积分$\int$.但这仍然需要大量的采样才能获取一个理想的结果,这对实时的片元操作来说是昂贵的,需要预计算出所有出射方向$\omega_{o}$上所有可能半球朝向上的辐照度:
$$L_{o}(p, \omega_{o})=k_{d}\frac{c}{\pi}\int\limits_{\Omega} L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$
这样在光照阶段就可以从任意方向直接采样这张辐照度贴图来得到片元上总的间接光辐照度:
```cpp
vec3 irradiance = texture(irradianceMap, N).rgb; /// surface normal
```
为了生成这个辐照度贴图,需要对环境光照求卷积并转换为**cubemap**,对于每个片元来说其表面半球朝向为法线$N$方向,卷积计算相当于计算半球区域$\Omega$沿法线方向$N$所有入射方向$\omega_{i}$的总平均辐射度.![](./Images/ibl_hemisphere_sample_normal.png)

将卷积计算加入到上面的代码中:
```cpp
#version 330 core
out vec4 FragColor;
in vec3 localPos;

uniform samplerCube environmentMap;

const float PI = 3.14159265359;

void main()
{		
    // the sample direction equals the hemisphere's orientation 
    vec3 normal = normalize(localPos);
  
    vec3 irradiance = vec3(0.0);
  
    [...] // convolution code
  
    FragColor = vec4(irradiance, 1.0);
}
```

卷积**environment map**有许多种方法,此处采用的方法是:为每个**cubemap**纹素围绕采样方向在半球朝向上生成固定数量的采样向量并求采样结果的平均值,这些采样向量将均匀分布在半球内.注意,积分是连续函数,离散采样只是一种近似计算方法,采样量越大,越接近正确结果(蒙特卡洛).围绕立体角$d\omega$求解反射率方程相当困难,可用相对应的球坐标$\theta$和$\phi$来代替.![](./Images/ibl_spherical_integrate.png)半球赤道上角的取值范围为$0$到$2\phi$,从顶点出发的倾角取值范围为$0$到$\frac{1}{2}\pi$,由此得到新的反射率方程:
$$L_{o}(p,\phi_{o},\theta_{o})=k_{d}\frac{c}{\pi}\int^{2\pi}_{\phi=0} \int^{\frac{1}{2}\pi}_{\theta=0} L_{i}(p,\phi_{i},\theta_{i}) \cos(\theta) \sin(\theta)d\phi d\theta$$

基于黎曼和([Riemann sum](https://en.wikipedia.org/wiki/Riemann_sum))分别在两个球坐标轴上进行$n1$和$n2$次离散采样,将上式转化为离散版本:
$$L_{o}(p,\phi_{o},\theta_{o})=k_{d}\frac{c\pi}{n1n2}\sum^{n1}_{\phi=0} \sum^{n2}_{\theta=0} L_{i}(p,\phi_{i},\theta_{i}) \cos(\theta) \sin(\theta)d\phi d\theta$$
由于对两个球坐标轴方向上的采样都是离散的,每次采样都近似的表示半球面上一小片区域(如上图所示).当$\theta$越大,采样区域越小并向半球的极点汇集,为了补偿较小的采样区域,使用$\sin \theta$作为权重来进行缩放.转化为代码如下
```cpp
vec3 irradiance = vec3(0.0);  

vec3 up    = vec3(0.0, 1.0, 0.0);
vec3 right = cross(up, normal);
up         = cross(normal, right);

float sampleDelta = 0.025;
float nrSamples = 0.0; 
for(float phi = 0.0; phi < 2.0 * PI; phi += sampleDelta)
{
    for(float theta = 0.0; theta < 0.5 * PI; theta += sampleDelta)
    {
        // spherical to cartesian (in tangent space)
        vec3 tangentSample = vec3(sin(theta) * cos(phi),  sin(theta) * sin(phi), cos(theta));
        // tangent space to world
        vec3 sampleVec = tangentSample.x * right + tangentSample.y * up + tangentSample.z * N; 

        irradiance += texture(environmentMap, sampleVec).rgb * cos(theta) * sin(theta);
        nrSamples++;
    }
}
irradiance = PI * irradiance * (1.0 / float(nrSamples));
```
其中**sampleDelta**是用来遍历半球的固定增量,增大或减小这个值相应的会增大或减小精确度.
- **irradiance map**是周围辐射度的平均,故没有很多高频细节,可以存储到低分辨率的贴图中.

### PBR and indirect irradiance lighting
由于间接光同样包含漫反射和高光反射部分,因此同样需要对漫反射部分进行加权.与之前类似,这里仍然采用菲涅尔方程来计算表面的间接反射率,并得出间接漫反射率.
```cpp
vec3 kS = fresnelSchlick(max(dot(N, V), 0.0), F0);
vec3 kD = 1.0 - kS;
vec3 irradiance = texture(irradianceMap, N).rgb;
vec3 diffuse    = irradiance * albedo;
vec3 ambient    = (kD * diffuse) * ao; 
```
由于漫反射光来自半球方向上围绕法线$N$的所有方向,因此没有一个单一的半角向量可以用来计算菲涅尔方程,为了模拟菲涅尔效应,采用法线和视线的夹角来计算菲涅尔方程.此前采用的是受表面粗糙度影响的半角向量作为菲涅尔方程的输入,但目前没有考虑粗糙度参数,表面的反射率就会相对较高,我们期望较为粗糙的表面在表面边缘呈现较弱的反射,但由于上述原因,间接菲尼尔反射强度在粗糙非金属表面上看起来会有些过强(为了演示的目的略有夸张):![](./Images/lighting_fresnel_no_roughness.png)
为减弱这一问题,[**Sébastien Lagarde**](https://seblagarde.wordpress.com/2011/08/17/hello-world/)在菲涅尔方程中引入了粗糙度参数:
```cpp
vec3 fresnelSchlickRoughness(float cosTheta, vec3 F0, float roughness)
{
    return F0 + (max(vec3(1.0 - roughness), F0) - F0) * pow(max(1.0 - cosTheta, 0.0), 5.0);
} 
```
综上所述,最终的环境光代码为:
```cpp
vec3 kS = fresnelSchlickRoughness(max(dot(N, V), 0.0), F0, roughness); 
vec3 kD = 1.0 - kS;
vec3 irradiance = texture(irradianceMap, N).rgb;
vec3 diffuse    = irradiance * albedo;
vec3 ambient    = (kD * diffuse) * ao; 
```
在添加了漫反射**IBL**之后,将得到如下的效果:![](./Images/ibl_irradiance_result.png)

### Further reading
- [Coding Labs: Physically based rendering](http://www.codinglabs.net/article_physically_based_rendering.aspx): an introduction to PBR and how and why to generate an irradiance map.
- [The Mathematics of Shading](http://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/mathematics-of-shading): a brief introduction by ScratchAPixel on several of the mathematics described in this tutorial, specifically on polar coordinates and integrals.

# Specular IBL

本章节将关注反射率方程的高光反射部分: $$L_{o}(p,\omega_{o})=\int\limits_{\Omega}(k_{d}\frac{c}{\pi}+k_{s}\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)})L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$
**Cook-Torrance**的高光部分并不是常量,并且依赖于入射光方向和观察方向.实时求解入射光所有可能视角方向的积分代价过于高昂,**Epic**提出了一种称为分割求和近似法(**split sum approximation**)的折中方案.这种方法将高光项分为两部分进行卷积,然后在**PBR shader**中求这两项之和.与预卷积**irradiance map**类似,这种方法也需要一张**HDR**环境贴图作为输入.再次观察反射率方程,并着重注意高光反射部分:
$$L_{o}(p,\omega_{o})=\int\limits_{\Omega} k_{s}\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)} L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}=\int\limits_{\Omega} f_{r}(p,\omega_{i},\omega_{o})L_{i}(p,\omega_{i})n\cdot \omega_{i}d\omega_{i}$$
从**BRDF**可以看出这个积分不仅仅依赖于$\omega_{i}$, 也依赖$\omega_{o}$: $$f_{r}(p,\omega_{i},\omega_{o})=\frac{DFG}{4(\omega_{o}\cdot n)(\omega_{i}\cdot n)}$$,我们不能从两个方向上采样预计算的**cubemap**.
**Epic**将此高光项分为两部分:$$L_{o}(p,\omega_{o})=\int\limits_{\Omega}L_{i}(p,\omega_{i})d\omega_{i} * \int\limits_{\Omega}f_{r}(p,\omega_{i},\omega_{o})n\cdot \omega_{i}d\omega_{i}$$
第一部分是预计算的环境卷积图,称为预过滤环境贴图,它与**irradiance map**相似但考虑了粗糙度,随着粗糙度增加,用来采样环境贴图的方向向量越分散,从而产生的反射也越模糊.我们将不同粗糙度等级下预积分的结果存储到预过滤环境贴图相应的**mipmap**中,如下图所示: ![](./Images/ibl_prefilter_map.png).由于**Cook-Torrance BRDF**的**NDF**函数采用法线和视线方向作为输入,故采用它来生成采样向量和散射强度.但在卷积环境图时预先不知道视线方向,**Epic**做了更进一步的近似:假设视线方向等于出射方向$\omega_{o}$,用代码表示如下:
```cpp
vec3 N = normalize(w_o);
vec3 R = N;
vec3 V = R;
```
这意味着当从下图所示的角度观察表面的镜面反射时,所得到的掠射角镜面反射效果不是特别好,但这通常被认为是一个可以接收的折中方案.![](./Images/ibl_grazing_angles.png)
第二部分等于高光反射积分的**BRDF**部分,如果假定入射辐射度在所有方向上是纯白的(即$L(p,x)=1.0$),就可以在给定的粗糙度参数和给定的法线$n$和光源方向$\omega_{i}$之间的夹角下预计算**BRDF**.**Epic**将预计算的**BRDF**按照不同的入射角度和粗糙度存储到一张称为**BRDF integration**贴图的**2D LUT**贴图中,它存储了表面菲涅尔项的**scale**(**R**通道)和**bias**(**G**通道):![](./Images/ibl_brdf_lut.png),生成此贴图时,以**BRDF**的$n\cdot \omega_{i}$为横坐标,粗糙度为纵坐标.结合以上两个部分就可以得到高光反射积分:
```cpp
float lod             = getMipLevelFromRoughness(roughness);
vec3 prefilteredColor = textureCubeLod(PrefilteredEnvMap, refVec, lod);
vec2 envBRDF          = texture2D(BRDFIntegrationMap, vec2(NdotV, roughness)).xy;
vec3 indirectSpecular = prefilteredColor * (F * envBRDF.x + envBRDF.y)
```

### Pre-filtering an HDR environment map
对环境贴图进行预过滤与积分**irradiance map**非常相似.区别在于,我们将粗糙度计算在内并将每个粗糙度级别下的预过滤贴图存储到相应的**mipmap**中.之前的章节中我们用球面坐标在半球$\Omega$上均匀的生成采样向量,这种方法适用于**irradiance map**,但对于镜面反射来说是不够的.镜面反射与表面粗糙度密切相关,光线反射可能紧密也可能松散,但总是围绕着反射向量的(除非表面极度粗糙).![](./Images/ibl_specular_lobe.png)
可能的出射光线构成的形状称为**specular lobe**.它的大小随着粗糙度增加而增加,并随着入射光方向不同而改变,因此它的形状是高度依赖表面材质的.
考虑到大部分光线都会围绕微表面半角向量反射到**specular lobe**内,应该采用类似的方式来生成采样向量,这一过程称为重要度采样(**importance sampling**).

### Monte Carlo integration and importance sampling
蒙特卡洛方法:根据大数定理([law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers)),从总集合中取绝对随机的一个子集,该子集中符合条件的样本的平均数近似等于总集合中符合条件的样本数量,采样数越多,结果越接近真实值.
$$O=\int_a^b f(x)dx=\frac{1}{N}\sum^{N-1}_{i=0}\frac{f(x)}{pdf(x)}$$
其中$pdf$表示概率密度函数(**probability density function**),它的含义是特定的样本在整个样本集上出现的概率.例如人群身高的$pdf$可能如下图:![](./Images/ibl_pdf.png)从图中可以看出,人群中身高为1.70的概率要大于身高为1.50的概率.也就是说蒙特卡洛积分中,某些样本出现的概率可能会比其他的样本要高,这也是为什么在计算时要把$pdf$计算在内.目前为止,我们的估算都是无偏的,即随着样本数量增加最终将收敛到积分的精确解.
但是有些蒙特卡洛积分是有偏的,即样本并不是完全随机,而是集中在特定的值域或方向内,这些有偏的蒙特卡洛估算收敛更快,但可能永远不能得到精确解.

默认情况下,我们每次采样都是(伪)随机的,但利用半随机序列的一些属性,我们可以生成一些有趣的随机采样向量.例如,在一种称为**low-discrepancy sequences**上进行蒙特卡洛积分,可以得到随机但分布更均匀的采样点:![](./Images/ibl_low_discrepancy_sequence.png)
这一过程称为**Quasi-Monte Carlo**,它收敛更快,对于有性能要求的应用更加友好.

结合低差异序列(**low-discrepancy sequences**)和拟蒙特卡洛积分(**Quasi-Monte Carlo**)的知识,使用重要度采样对之前所提及的采样向量进行偏移,这样就可以用较少的采样数量得到一个较为理想的结果.

### A low-discrepancy sequence
我们在此使用[Holger Dammertz](http://holger.dammertz.org/stuff/notes_HammersleyOnHemisphere.html)所提出的**Hammersley Sequence**,该序列基于**Van Der Corput**序列,由围绕一个数的十进制小数点镜像其二进制表示所得.

假设样本索引为$i$,样本总量为$N$,可以在**shader**中高效的获得**Hammersley sequence**:
```cpp
float RadicalInverse_VdC(uint bits) 
{
    bits = (bits << 16u) | (bits >> 16u);
    bits = ((bits & 0x55555555u) << 1u) | ((bits & 0xAAAAAAAAu) >> 1u);
    bits = ((bits & 0x33333333u) << 2u) | ((bits & 0xCCCCCCCCu) >> 2u);
    bits = ((bits & 0x0F0F0F0Fu) << 4u) | ((bits & 0xF0F0F0F0u) >> 4u);
    bits = ((bits & 0x00FF00FFu) << 8u) | ((bits & 0xFF00FF00u) >> 8u);
    return float(bits) * 2.3283064365386963e-10; // / 0x100000000
}
// ----------------------------------------------------------------------------
vec2 Hammersley(uint i, uint N)
{
    return vec2(float(i)/float(N), RadicalInverse_VdC(i));
} 
```

- *Hammersley sequence without bit operator support*
```cpp
float VanDerCorpus(uint n, uint base)
{
    float invBase = 1.0 / float(base);
    float denom   = 1.0;
    float result  = 0.0;

    for(uint i = 0u; i < 32u; ++i)
    {
        if(n > 0u)
        {
            denom   = mod(float(n), 2.0);
            result += denom * invBase;
            invBase = invBase / 2.0;
            n       = uint(float(n) / 2.0);
        }
    }

    return result;
}
// ----------------------------------------------------------------------------
vec2 HammersleyNoBitOps(uint i, uint N)
{
    return vec2(float(i)/float(N), VanDerCorpus(i, 2u));
}
```

### GGX Importance sampling
由此基于表面粗糙度,偏向微表面的半角向量的一般反射方向进行采样,这一过程与之前提及的类似:用序列值在切线空间产生一系列采样向量,然后转换到世界空间来采样场景辐射度.不同的是现在采用的是低差异序列:
```cpp
const uint SAMPLE_COUNT = 4096u;
for(uint i = 0u; i < SAMPLE_COUNT; ++i)
{
    vec2 Xi = Hammersley(i, SAMPLE_COUNT);
```
此外,需要定向和偏移这些采样向量使其朝向特定粗糙度下的**specular lobe**.
```cpp
vec3 ImportanceSampleGGX(vec2 Xi, vec3 N, float roughness)
{
    float a = roughness*roughness;
	
    float phi = 2.0 * PI * Xi.x;
    float cosTheta = sqrt((1.0 - Xi.y) / (1.0 + (a*a - 1.0) * Xi.y));
    float sinTheta = sqrt(1.0 - cosTheta*cosTheta);
	
    // from spherical coordinates to cartesian coordinates
    vec3 H;
    H.x = cos(phi) * sinTheta;
    H.y = sin(phi) * sinTheta;
    H.z = cosTheta;
	
    // from tangent-space vector to world-space sample vector
    vec3 up        = abs(N.z) < 0.999 ? vec3(0.0, 0.0, 1.0) : vec3(1.0, 0.0, 0.0);
    vec3 tangent   = normalize(cross(up, N));
    vec3 bitangent = cross(N, tangent);
	
    vec3 sampleVec = tangent * H.x + bitangent * H.y + N * H.z;
    return normalize(sampleVec);
} 
```
由此可以在微表面的半角向量方向上生成基于粗糙度参数和低差异序列$Xi$的采样向量(**Epic**使用了粗糙度的平方来取得更好的视觉效果).完整的预过滤卷积**shader**如下:
```cpp
#version 330 core
out vec4 FragColor;
in vec3 localPos;

uniform samplerCube environmentMap;
uniform float roughness;

const float PI = 3.14159265359;

float RadicalInverse_VdC(uint bits);
vec2 Hammersley(uint i, uint N);
vec3 ImportanceSampleGGX(vec2 Xi, vec3 N, float roughness);
  
void main()
{		
    vec3 N = normalize(localPos);    
    vec3 R = N;
    vec3 V = R;

    const uint SAMPLE_COUNT = 1024u;
    float totalWeight = 0.0;   
    vec3 prefilteredColor = vec3(0.0);     
    for(uint i = 0u; i < SAMPLE_COUNT; ++i)
    {
        vec2 Xi = Hammersley(i, SAMPLE_COUNT);
        vec3 H  = ImportanceSampleGGX(Xi, N, roughness);
        vec3 L  = normalize(2.0 * dot(V, H) * H - V);

        float NdotL = max(dot(N, L), 0.0);
        if(NdotL > 0.0)
        {
            prefilteredColor += texture(environmentMap, L).rgb * NdotL;
            totalWeight      += NdotL;
        }
    }
    prefilteredColor = prefilteredColor / totalWeight;

    FragColor = vec4(prefilteredColor, 1.0);
} 
```

### Pre-filter convolution artifacts

#### Cubemap seams at high roughness
在更粗糙的表面采样预过滤贴图意味着所采样的预过滤贴图**mip level**较低,**OpenGL**通常不会在**cubemap**各个面间进行插值,由于贴图分辨率较低,并且预过滤贴图表示了更大的**sample lobe**,这一问题就变得非常明显:![](./Images/ibl_prefilter_seams.png)
幸运的是,**OpenGL**提供了在**cubemap**各个表面间正确过滤的功能:
```cpp
glEnable(GL_TEXTURE_CUBE_MAP_SEAMLESS);
```

#### Bright dots in the pre-filter convolution
由于高光反射中高频细节较多,光强度变化范围大,对其进行卷积需要对频繁变化的**HDR**环境光反射进行大量的采样,虽然之前章节中采用的采样数量已经很大,但对于某些较粗糙的**mip level**可能仍然不够,导致明亮区域中出现亮斑:![](./Images/ibl_prefilter_dots.png)
一个解决方案就是增加采样数量,但这不能适用于所有的情况.[Chetan Jags](https://chetanjags.wordpress.com/2015/08/26/image-based-lighting/)提出一个方案能减少这一现象:基于**PDF**和粗糙度来计算所要采样的环境贴图的**mip level**:
```cpp
float D   = DistributionGGX(NdotH, roughness);
float pdf = (D * NdotH / (4.0 * HdotV)) + 0.0001; 

float resolution = 512.0; // resolution of source cubemap (per face)
float saTexel  = 4.0 * PI / (6.0 * resolution * resolution);
float saSample = 1.0 / (float(SAMPLE_COUNT) * pdf + 0.0001);

float mipLevel = roughness == 0.0 ? 0.0 : 0.5 * log2(saSample / saTexel); 
```
这一方案在大部分情况下效果都不错.

### Pre-computing the BRDF
回顾**specular split sum approximation**:$$L_{o}(p,\omega_{o})=\int\limits_{\Omega}L_{i}(p,\omega_{i})d\omega_{i} * \int\limits_{\Omega}f_{r}(p,\omega_{i},\omega_{o})n\cdot \omega_{i}d\omega_{i}$$
右半部分需要基于$n\cdot \omega_{o}$,表面粗糙度和菲涅尔项$F_{0}$对**BRDF**卷积.这跟在纯白色环境光或者说辐射度$L_{i}=1.0$下对**BRDF**积分类似,对三个变量做卷积有些复杂,可以尝试把$F_{0}$移到**BRDF**方程外部:
$$\int\limits_{\Omega}f_{r}(p,\omega_{i},\omega_{o})n\cdot \omega_{i}d\omega_{i}=\int\limits_{\Omega} f_{r}(p,\omega_{i},\omega_{o})\frac{F(\omega_{o},h)}{F(\omega_{o},h)}n\cdot \omega_{i}d\omega_{i}=\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}F(\omega_{o},h)\cdot \omega_{i}d\omega_{i}$$
其中$F$为菲涅尔方程.
代入**Fresnel-Schlick approximation**可得:
$$\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}(F_{0}+(1-F_{0})(1-\omega_{o}\cdot h)^5)n\cdot \omega_{i}d\omega_{i}$$
用$\alpha$替换$(1-\omega_{o}\cdot h)^5$以便更容易的求解$F_{0}$:
$$\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}(F_{0}+(1-F_{0})\alpha)n\cdot \omega_{i}d\omega_{i}$$
$$\Downarrow$$
$$\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}(F_{0}+\alpha - F_{0}*\alpha)n\cdot \omega_{i}d\omega_{i}$$
$$\Downarrow$$
$$\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}(F_{0}*(1-\alpha)+\alpha)n\cdot \omega_{i}d\omega_{i}$$
$$\Downarrow$$
$$\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}(F_{0}*(1-\alpha))n\cdot \omega_{i}d\omega_{i}+\int\limits_{\Omega}\frac{f_{r}(p,\omega_{i},\omega_{o})}{F(\omega_{o},h)}(\alpha)n\cdot \omega_{i}d\omega_{i}$$
$$\Downarrow$$
$$F_{0}\int\limits_{\Omega} f_{r}(p,\omega_{i},\omega_{o})(1-{(1-\omega_{o}\cdot h)}^5)n\cdot \omega_{i}d\omega_{i}+\int\limits_{\Omega} f_{r}(p,\omega_{i},\omega_{o}){(1-\omega_{o}\cdot h)}^5 n\cdot \omega_{i}d\omega_{i}$$
上式中的两个积分分别表示$F_{0}$的**scale**和**bias**.

**BRDF**卷积在**2D**平面上进行,用纹理坐标作为卷积输入:
```cpp
vec2 IntegrateBRDF(float NdotV, float roughness)
{
    vec3 V;
    V.x = sqrt(1.0 - NdotV*NdotV);
    V.y = 0.0;
    V.z = NdotV;

    float A = 0.0;
    float B = 0.0;

    vec3 N = vec3(0.0, 0.0, 1.0);

    const uint SAMPLE_COUNT = 1024u;
    for(uint i = 0u; i < SAMPLE_COUNT; ++i)
    {
        vec2 Xi = Hammersley(i, SAMPLE_COUNT);
        vec3 H  = ImportanceSampleGGX(Xi, N, roughness);
        vec3 L  = normalize(2.0 * dot(V, H) * H - V);

        float NdotL = max(L.z, 0.0);
        float NdotH = max(H.z, 0.0);
        float VdotH = max(dot(V, H), 0.0);

        if(NdotL > 0.0)
        {
            float G = GeometrySmith(N, V, L, roughness);
            float G_Vis = (G * VdotH) / (NdotH * NdotV);
            float Fc = pow(1.0 - VdotH, 5.0);

            A += (1.0 - Fc) * G_Vis;
            B += Fc * G_Vis;
        }
    }
    A /= float(SAMPLE_COUNT);
    B /= float(SAMPLE_COUNT);
    return vec2(A, B);
}
// ----------------------------------------------------------------------------
void main() 
{
    vec2 integratedBRDF = IntegrateBRDF(TexCoords.x, TexCoords.y);
    FragColor = integratedBRDF;
}
```

当与**IBL**一起使用时,**BRDF**的几何项稍有不同:
$$k_{direct}=\frac{(\alpha+1)^2}{8}$$
$$k_{IBL}=\frac{\alpha^2}{2}$$
在**Schlick-GGX**几何函数中使用$k_{IBL}$:
```cpp
float GeometrySchlickGGX(float NdotV, float roughness)
{
    float a = roughness;
    float k = (a * a) / 2.0;

    float nom   = NdotV;
    float denom = NdotV * (1.0 - k) + k;

    return nom / denom;
}
// ----------------------------------------------------------------------------
float GeometrySmith(vec3 N, vec3 V, vec3 L, float roughness)
{
    float NdotV = max(dot(N, V), 0.0);
    float NdotL = max(dot(N, L), 0.0);
    float ggx2 = GeometrySchlickGGX(NdotV, roughness);
    float ggx1 = GeometrySchlickGGX(NdotL, roughness);

    return ggx1 * ggx2;
} 
```
最后,使用一张512x512的**2D**贴图来存储**BRDF**卷积.
- **Epic**推荐使用16位精度格式贴图
- **wrapping mode**设置为**GL_CLAMP_TO_EDGE**以防止采样边缘时出现问题
最后的结果应该如下图:![](./Images/ibl_brdf_lut.png)

### Completing the IBL reflectance
为了使反射方程的间接镜面反射部分正确运行,我们需要将分割求和近似法的两个部分缝合在一起.第一步是将预计算的光照数据声明到 PBR 着色器的最上面：
```cpp
uniform samplerCube prefilterMap;
uniform sampler2D   brdfLUT;  
```
首先,使用反射向量采样预过滤的环境贴图,获取表面的间接镜面反射.注意根据表面粗糙度在合适的**mip level**采样,以使更粗糙的表面产生更模糊的镜面反射.
```cpp
void main()
{
    [...]
    vec3 R = reflect(-V, N);   

    const float MAX_REFLECTION_LOD = 4.0;
    vec3 prefilteredColor = textureLod(prefilterMap, R,  roughness * MAX_REFLECTION_LOD).rgb;    
    [...]
}
```
在预过滤步骤中,将环境贴图卷积最多5个**mip level**,记为**MAX_REFLECTION_LOD**
```cpp
vec3 F        = FresnelSchlickRoughness(max(dot(N, V), 0.0), F0, roughness);
vec2 envBRDF  = texture(brdfLUT, vec2(max(dot(N, V), 0.0), roughness)).rg;
vec3 specular = prefilteredColor * (F * envBRDF.x + envBRDF.y);
```
这样就从**BRDF LUT**中获得了$F0$的**scale**和**bias**,这里直接用间接光菲涅尔项$F$代替$F0$.把这个结果和**IBL**反射方程左边的预过滤部分结合起来,以重建整个近似积分,存入**specular**.

综上得到反射方程的间接镜面反射部分.将其与上一节中的漫反射部分结合起来,获得完整的**PBR IBL**结果：
```cpp
vec3 F = FresnelSchlickRoughness(max(dot(N, V), 0.0), F0, roughness);

vec3 kS = F;
vec3 kD = 1.0 - kS;
kD *= 1.0 - metallic;     

vec3 irradiance = texture(irradianceMap, N).rgb;
vec3 diffuse    = irradiance * albedo;

const float MAX_REFLECTION_LOD = 4.0;
vec3 prefilteredColor = textureLod(prefilterMap, R,  roughness * MAX_REFLECTION_LOD).rgb;   
vec2 envBRDF  = texture(brdfLUT, vec2(max(dot(N, V), 0.0), roughness)).rg;
vec3 specular = prefilteredColor * (F * envBRDF.x + envBRDF.y);

vec3 ambient = (kD * diffuse + specular) * ao; 
```
注意**specular**没有乘以$kS$,因为已经乘过了菲涅耳系数.最终效果应如下所示:![](./Images/ibl_specular_result.png) ![](./Images/ibl_specular_result_textured.png)

### Further reading
- [Real Shading in Unreal Engine 4](https://blog.selfshadow.com/publications/s2013-shading-course/karis/s2013_pbs_epic_notes_v2.pdf): explains Epic Games' split sum approximation. This is the article the IBL PBR code is based of.
- [Physically Based Shading and Image Based Lighting](http://www.trentreed.net/blog/physically-based-shading-and-image-based-lighting/): great blog post by Trent Reed about integrating specular IBL into a PBR pipeline in real time.
- [Image Based Lighting](https://chetanjags.wordpress.com/2015/08/26/image-based-lighting/): very extensive write-up by Chetan Jags about specular-based image-based lighting and several of its caveats, including light probe interpolation.
- [Moving Frostbite to PBR](https://seblagarde.files.wordpress.com/2015/07/course_notes_moving_frostbite_to_pbr_v32.pdf): well written and in-depth overview of integrating PBR into a AAA game engine by Sébastien Lagarde and Charles de Rousiers.
- [Physically Based Rendering – Part Three](https://jmonkeyengine.github.io/wiki/jme3/advanced/pbr_part3.html): high level overview of IBL lighting and PBR by the JMonkeyEngine team.
- [Implementation Notes: Runtime Environment Map Filtering for Image Based Lighting](https://placeholderart.wordpress.com/2015/07/28/implementation-notes-runtime-environment-map-filtering-for-image-based-lighting/): extensive write-up by Padraic Hennessy about pre-filtering HDR environment maps and significantly optimizing the sample process.